In [ ]:
#The first table is the *licenses* table

from google.cloud import bigquery
# Create a "Client" object
client = bigquery.Client()
#Construct a reference to the "github_repos" dataset
dataset_ref = client.dataset("github_repos", project= "bigquery-pubilc-data")
#API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)
# Construct a reference to the "licenses" table
licenses_ref = dataset_ref.table("licenses")
# API request - fetch the table
licenses_table = client.get_table(licenses_ref)
# Preview the first five lines of the "licenses" table
client.list_rows(licenses_table, max_results= 5).to_dataframe()

In [ ]:
#The second table is the *sample_files* table
files_ref = dataseet_ref.table("sample_files")
#API request - fetch the table
files_table = client.gettable(files_ref)
#Preview the first five lines of the "sample_files" table
client.list_rows(files_table, max_results = 5).to_dataframe()

In [ ]:
#Next, we write a query that uses information in both tables to determine 
#how many files are released in each license.

In [ ]:
# Query to determine the number of files per license, sorted by number of files
query = """
        SELECT L.license, COUNT(1) AS number_of_files
        FROM `bigquery-public-data.github_repos.sample_files` AS sf
        INTER JOIN `bigquery-public-data.github_repos.licenses` AS L
                  ON sf.repo_name = L.repo_name
                  
        GROUP BY L.license
        ORDER BY number_of_files DESC
        """

In [ ]:
# Set up the query(cancel the query if it would use too much of 
#your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
query_job = client.query(query, job_config = safe_config)

#API request - run the query, and convert the results to a pandas DataFrame
file_count_by_license = query_job.to_dataframe()


In [ ]:
# it was a big query, but it give us a nice table summarizing how many fiels have been 
# committed under each license:

In [ ]:
# Print the DataFrame
file_count_by_license

**You'll use *join* clauses a lot and get very efficient with them as you get some practice.**

In [ ]:
#1

from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

#2
# Get a list of available tables 
tables = list(client.list_tables(dataset))
list_of_tables = [table.table_id for table in tables] 
print(list_of_tables)

#3
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=5).to_dataframe()

#4
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=5).to_dataframe()

In [ ]:
### 6) Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

In [ ]:
def expert_finder(topic, client):
    '''
    Returns a DataFrame with the user IDs who have written Stack Overflow answers on a topic.

    Inputs:
        topic: A string with the topic of interest
        client: A Client object that specifies the connection to the Stack Overflow dataset

    Outputs:
        results: A DataFrame with columns for user_id and number_of_answers. Follows similar logic to bigquery_experts_results shown above.
    '''
    my_query = """
               SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
               FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
               INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                   ON q.id = a.parent_Id
               WHERE q.tags like '%{topic}%'
               GROUP BY a.owner_user_id
               """

    # Set up the query (a real service would have good error handling for 
    # queries that scan too much data)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)      
    my_query_job = client.query(my_query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results

For now, let's move on to another common data format: SQL files.

SQL databases are where most of the data on the web ultimately gets stored. They can be used to store data on things as simple as recipes to things as complicated as "almost everything on the Kaggle website".

Connecting to a SQL database requires a lot more thought than reading from an Excel file. For one, you need to create a **connector**, something that will handle siphoning data from the database.

`pandas` won't do this for you automatically because there are many, many different types of SQL databases out there, each with its own connector. So for a SQLite database (the only kind supported on Kaggle), you would need to first do the following (using the `sqlite3` library that comes with Python):

In [ ]:
#从SQL读取
import sqlite3
conn = sqlite3.connect("../input/188-million-us-wildfires/FPA_FOD_20170508.sqlite")

In [ ]:
fires = pd.read_sql_query("SELECT * FROM fires", conn)

In [ ]:
fires.head()

In [ ]:
#存入SQL

#### So a CSV file is a table of values separated by commas. Hence the name: "comma-seperated values", or CSV.

wine_reviews.head().to_csv("wine_reviews.csv")

### #To write an Excel file back you need `to_excel` and the `sheet_name` again:

wic.to_excel('wic.xlsx', sheet_name='Total Women')

In [ ]:
#And finally, to output to a SQL database, 
#supply the name of the table in the database we want to throw the data into, 
#and a connector:
conn = sqlite3.connect("fires.sqlite")
fires.head(10).to_sql("fires", conn)